In [2]:
import speech_recognition as sr
from pydub import AudioSegment
from docx import Document
from tkinter import Tk, filedialog, simpledialog, messagebox
import os

def transcrever_audio_segmento(arquivo_segmento):
    """
    Transcreve um segmento de áudio usando a API do Google.

    Args:
        arquivo_segmento (str): Caminho do arquivo de áudio a ser transcrito.

    Returns:
        str: Texto transcrito ou mensagem de erro.
    """
    recognizer = sr.Recognizer()
    with sr.AudioFile(arquivo_segmento) as source:
        audio_data = recognizer.record(source)
    try:
        texto = recognizer.recognize_google(audio_data, language='pt-BR')
        return texto
    except sr.UnknownValueError:
        return "O serviço de reconhecimento de fala não conseguiu entender o áudio"
    except sr.RequestError as e:
        return f"Erro ao solicitar resultados do serviço de reconhecimento de fala; {e}"

def dividir_audio(arquivo_audio, duracao_segmento_ms):
    """
    Divide um arquivo de áudio em segmentos de duração especificada.

    Args:
        arquivo_audio (str): Caminho do arquivo de áudio a ser dividido.
        duracao_segmento_ms (int): Duração de cada segmento em milissegundos.

    Returns:
        list: Lista de caminhos dos arquivos de segmento gerados.
    """
    audio = AudioSegment.from_file(arquivo_audio)  # Alterado para aceitar diferentes formatos
    segmentos = []
    for i in range(0, len(audio), duracao_segmento_ms):
        segmento = audio[i:i + duracao_segmento_ms]
        segmento_filename = f"segmento_{i//duracao_segmento_ms}.wav"
        segmento.export(segmento_filename, format="wav")
        segmentos.append(segmento_filename)
    return segmentos

def salvar_transcricao_no_word(transcricao, nome_arquivo_word):
    """
    Salva a transcrição em um arquivo Word.

    Args:
        transcricao (str): Texto a ser salvo.
        nome_arquivo_word (str): Nome do arquivo Word a ser criado.
    """
    doc = Document()
    doc.add_paragraph(transcricao)
    doc.save(nome_arquivo_word)
    print(f"Arquivo Word salvo como: {nome_arquivo_word}")

def transcrever_audio_longos(arquivo_audio, duracao_segmento_ms=60000):
    """
    Transcreve um arquivo de áudio longo dividindo-o em segmentos menores.

    Args:
        arquivo_audio (str): Caminho do arquivo de áudio a ser transcrito.
        duracao_segmento_ms (int): Duração dos segmentos em milissegundos.

    Returns:
        str: Transcrição completa do áudio.
    """
    segmentos = dividir_audio(arquivo_audio, duracao_segmento_ms)
    transcricao_completa = ""

    for segmento in segmentos:
        transcricao_segmento = transcrever_audio_segmento(segmento)
        transcricao_completa += transcricao_segmento + "\n"
        print(f"Transcrição do segmento {segmento}: {transcricao_segmento}")

        # Exclui o arquivo de áudio quebrado após a transcrição
        try:
            os.remove(segmento)
            print(f"Arquivo {segmento} excluído com sucesso.")
        except OSError as e:
            print(f"Erro ao excluir o arquivo {segmento}: {e}")

    return transcricao_completa

def selecionar_arquivo():
    """
    Abre uma janela para selecionar um arquivo de áudio.

    Returns:
        str: Caminho do arquivo de áudio selecionado.
    """
    root = Tk()
    root.withdraw()  # Esconde a janela principal
    arquivo_audio = filedialog.askopenfilename(
        title="Selecione o arquivo de áudio",
        filetypes=[("Arquivos de áudio", "*.wav *.mp3 *.flac *.ogg *.aiff")]
    )
    return arquivo_audio

def obter_nome_arquivo():
    """
    Solicita o nome do arquivo de saída ao usuário.

    Returns:
        str: Nome do arquivo fornecido pelo usuário.
    """
    root = Tk()
    root.withdraw()  # Esconde a janela principal
    nome_arquivo_word = simpledialog.askstring(
        "Nome do Arquivo",
        "Digite o nome do arquivo de saída (sem extensão):"
    )
    return nome_arquivo_word

def iniciar_transcricao():
    """
    Inicia o processo de transcrição do áudio selecionado e salva em um arquivo Word.
    """
    arquivo_audio = selecionar_arquivo()
    if arquivo_audio:
        print(f"Arquivo de áudio selecionado: {arquivo_audio}")
        nome_arquivo_word = obter_nome_arquivo()
        if nome_arquivo_word:
            if not nome_arquivo_word.endswith('.docx'):
                nome_arquivo_word += '.docx'
            print(f"Nome do arquivo de saída: {nome_arquivo_word}")
            transcricao = transcrever_audio_longos(arquivo_audio, duracao_segmento_ms=60000)
            print("Transcrição completa do áudio:", transcricao)
            salvar_transcricao_no_word(transcricao, nome_arquivo_word)
            print(f"A transcrição foi salva no arquivo: {nome_arquivo_word}")
        else:
            print("Nome do arquivo de saída não fornecido.")
            messagebox.showerror("Erro", "Nome do arquivo de saída não pode ser vazio.")
    else:
        print("Arquivo de áudio não selecionado.")
        messagebox.showerror("Erro", "Nenhum arquivo de áudio selecionado.")

# Iniciar o processo de transcrição
if __name__ == "__main__":
    iniciar_transcricao()


Arquivo de áudio selecionado: C:/Users/Romário/Downloads/WhatsApp Ptt 2024-10-16 at 20.08.09.ogg
Nome do arquivo de saída: teste 2.docx
Transcrição do segmento segmento_0.wav: hora do teste vamos fazer o teste meu amigo chat GPT vamos fazer o teste hahaha olha só como estamos indo no teste caralho vamos fazer teste teste do meu programa de transcrever Estamos testando testando testando o canal direito testando o canal esquerdo pois é agora Estamos testando o meu programa de transcrever meu amigo chat GPT galera do LinkedIn todos todos vão ficar ligadinhos ligadinhos ligadinhos olha só agora que tá com 30 segundos eu quero que o áudio tenha mais de um minuto porque eu preciso transcrever direitinho esse áudio foi tanta coisa que aconteceu tá um calor do caralho calor demais pois é acho que não vou receber no final de ano
Arquivo segmento_0.wav excluído com sucesso.
Transcrição do segmento segmento_1.wav: O serviço de reconhecimento de fala não conseguiu entender o áudio
Arquivo segmento